In [1]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
IMAGE_HEIGHT = 150
IMAGE_WIDTH = 150
BATCH_SIZE = 32

TRAIN_DIR = "E:/Datasets/IntelEnv/seg_train/seg_train"

In [3]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=0.15,
    subset='training',
    seed=124,
    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size = 32
)

val_data = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=0.15,
    subset='validation',
    seed=124,
    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size = 32
)

Found 14034 files belonging to 6 classes.
Using 11929 files for training.
Found 14034 files belonging to 6 classes.
Using 2105 files for validation.


In [5]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [9]:
base_model = InceptionV3(weights='imagenet', include_top=False)

In [10]:
base_model.trainable = False

In [11]:
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 3 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 3 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [20]:
inputs = tf.keras.Input(shape=(150, 150, 3))

In [21]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [22]:
x = data_augmentation(inputs)

In [23]:
x = base_model(x, training=False)

In [24]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
x = global_average_layer(x)

In [25]:
prediction_layer = tf.keras.layers.Dense(6)

In [26]:
outputs = prediction_layer(x)

In [27]:
model = tf.keras.Model(inputs, outputs)

In [28]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 150, 150, 3)       0         
_________________________________________________________________
inception_v3 (Functional)    (None, None, None, 2048)  21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 12294     
Total params: 21,815,078
Trainable params: 12,294
Non-trainable params: 21,802,784
_________________________________________________________________


In [29]:
model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [30]:

resp = model.fit(train_data, epochs=5)

Epoch 1/5
373/373 [==============================] - 51s 104ms/step - loss: 14.3316 - accuracy: 0.3465
Epoch 2/5
373/373 [==============================] - 38s 101ms/step - loss: 9.7889 - accuracy: 0.4034
Epoch 3/5
373/373 [==============================] - 40s 107ms/step - loss: 8.3068 - accuracy: 0.4300
Epoch 4/5
373/373 [==============================] - 39s 105ms/step - loss: 7.3677 - accuracy: 0.4372
Epoch 5/5
373/373 [==============================] - 39s 104ms/step - loss: 6.6429 - accuracy: 0.4496


In [32]:
base_model.layers